In [51]:
import pandas as pd
import numpy as np
import scipy

from scipy.stats import wilcoxon
from statsmodels.sandbox.stats.multicomp import multipletests 

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [3]:
aucs = pd.read_csv('h:\\MyProj\\Courcera\\Ml-mipt\\course4\\week3b\\work\\AUCs.txt', sep = '\t', header = 0)
y

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [6]:
sparse_aucs = aucs[['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']]
sparse_aucs.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,0.763,0.768,0.771,0.798
1,0.599,0.591,0.590,0.569
2,0.954,0.971,0.968,0.967
3,0.628,0.661,0.654,0.657
4,0.882,0.888,0.886,0.898


In [52]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_aucs.columns):
    for j, rhs_column in enumerate(sparse_aucs.columns):
        if i >= j:
            continue
        
        result = wilcoxon(sparse_aucs[lhs_column], sparse_aucs[rhs_column])
#        print(result)WilcoxonResult(statistic=131.5, pvalue=0.06291972262602667)
        corr_data.append([lhs_column, rhs_column, result.statistic, result.pvalue])

Wall time: 3.99 ms


In [53]:
aucs_correlation = pd.DataFrame.from_records(corr_data)
aucs_correlation.columns = ['auc_A', 'auc_B', 'stat', 'pvalue']

In [54]:
aucs_correlation.sort_values(by='stat', ascending=False)

,auc_A,auc_B,stat,pvalue
1,C4.5,C4.5+cf,43.0,0.861262
4,C4.5+m,C4.5+m+cf,22.0,0.327826
3,C4.5+m,C4.5+cf,17.0,0.046333
2,C4.5,C4.5+m+cf,11.0,0.015906
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
0,C4.5,C4.5+m,6.5,0.010757


In [55]:
(aucs_correlation.pvalue < 0.05).value_counts()

True     4
False    2
Name: pvalue, dtype: int64

In [56]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.pvalue, 
                                            alpha = 0.05, 
                                            method = 'holm') 
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject

In [57]:
aucs_correlation

,auc_A,auc_B,stat,pvalue,p_corrected,reject
0,C4.5,C4.5+m,6.5,0.010757,0.064543,False
1,C4.5,C4.5+cf,43.0,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,11.0,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,17.0,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,22.0,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,0.091636,False


In [58]:
aucs_correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [59]:
reject, p_corrected, a1, a2 = multipletests(aucs_correlation.pvalue, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
aucs_correlation['p_corrected'] = p_corrected
aucs_correlation['reject'] = reject

In [60]:
aucs_correlation.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64

In [47]:
aucs_correlation

,auc_A,auc_B,stat,pvalue,p_corrected,reject
0,C4.5,C4.5+m,87.5,0.322878,0.5,False
1,C4.5,C4.5+cf,97.5,0.500000,0.5,False
2,C4.5,C4.5+m+cf,86.5,0.306606,0.5,False
3,C4.5+m,C4.5+cf,88.5,0.339589,0.5,False
4,C4.5+m,C4.5+m+cf,97.5,0.500000,0.5,False
5,C4.5+cf,C4.5+m+cf,87.0,0.314696,0.5,False
